###
#
# BI-JUL: 1. Domácí úkol v B221
#
# Podrobný popis zadání naleznete na Course pages nebo ve svém repozitáři.
#


#
# Typ `Polynomial{T}`
#


Parametrický typ `Polynomial{T}`.
K vytvoření polynomu konstruktoru předáme pole:

    Polynomial([1, 2, 3])
q
Prvky v poly odpovídají koeficientům polynomu od nejnižšího stupně k největšímu.
Tj. volbě výše odpovídá polynom ``3x^2 + 2x + 1``.

Pole musí obsahovat alespoň jeden prvek, jinak nastane chyba.
Konstruktor správně určí stupeň zadaného polynomu (stupeň nulového polynomu
bereme jako ``-1``) a případně zahodí nepotřebné nulové koeficienty.

In [1]:
# pyright: reportMissingModuleSource=false

In [2]:

struct Polynomial{ T <: Number }
  coefficients::Vector{T}
  degree::Int64

  function Polynomial(coefficients::Vector{T}) where { T <: Number }
    degree = length(coefficients) - 1

    if degree == -1
      throw(ArgumentError("You have to provide at least one coefficient!"))
    end

    while last(coefficients) == zero(T) && degree > 0
      pop!(coefficients)
      degree -= 1
    end

    if length(coefficients) == 1 && first(coefficients) == zero(T)
      degree = -1
    end

    return new{T}(coefficients, degree)
  end
end

#### Porovnávání polynomů podle zvoleného datového typu.

In [3]:
import Base.==

function ==(p::Polynomial{T}, q::Polynomial{T}) where { T <: Number }
  # exact comparison
  p.coefficients == q.coefficients
end

function ==(p::Polynomial{T}, q::Polynomial{T}) where { T <: AbstractFloat }
  # approximate comparison
  p.coefficients ≈ q.coefficients
end

== (generic function with 173 methods)

#
#### Pěkná textová reprezentace polynomu.
#

Ukázka výstupu:

    julia> Polynomial([1, 2, 3])
    + 3*x^2 + 2*x^1 + 1

    julia> Polynomial([1, 0, 3//2])
    + 3//2*x^2 + 1//1

    julia> Polynomial([1])
    1

    julia> Polynomial([0])
    0


In [4]:
function Base.show(io::IO, p::Polynomial{T}) where { T <: Number }
    if p.degree <= 0
      return print(io, last(p.coefficients))
    end
  
    tokens = []
  
    for j in p.degree:-1:0
      c = p.coefficients[j + 1]
  
      if c < zero(T)
        append!(tokens, "- ", -c)
      elseif c > zero(T)
        append!(tokens, "+ ", c)
      end
  
      if j > 0 && c != zero(T)
        append!(tokens, "*x^", j, " ")
      end
    end
  
    print(io, tokens...)
  end

#### Algebraické operace `+` a `*`  

In [5]:
import Base.*, Base.+

Součin dvou polynomů.

In [6]:
function *(p::Polynomial{T}, q::Polynomial{T}) where { T <: Number }
  deg = 0 # degree of element of p which is iterated
  res = Polynomial([0]) # initialize result
  for i in p.coefficients
    tmp = zeros(T, deg) # fills 0's to positions which can't be accessed by the particular multiplication
    res += Polynomial(append!(tmp, [i*j for j in q.coefficients])) # multiply iterated element of p with all elements of q
    deg += 1
  end
  return res
end

* (generic function with 321 methods)

Součet dvou polynomů.

In [7]:
function +(p::Polynomial{T}, q::Polynomial{T}) where { T <: Number }
  x = length(p.coefficients) >= length(q.coefficients) ? p.coefficients : q.coefficients # choose maximum size array
  y = x == q.coefficients ? p.coefficients : q.coefficients # the minimum size array
  lx = length(x)
  ly = length(y)
  return Polynomial(append!([x[i]+y[i] for i in 1:ly], x[ly+1:end])) # sums common positions and append values of bigger polynom
end

+ (generic function with 207 methods)

#### Rozdíl dvou polynomů

In [8]:
import Base.-

In [9]:
function -(p::Polynomial{T}) where { T <: Number }
    return Polynomial([ -x for x in p.coefficients ])
end

- (generic function with 212 methods)

In [10]:
function -(p::Polynomial{T}, q::Polynomial{T} ) where { T <: Number }
    x = -q
    return p+x
end

- (generic function with 213 methods)

#
  #### Výpočet funkční hodnoty polynomu
  #
  
  Díky této metodě můžeme polynomy vyhodnocovat, tedy počítat jejich funkční hodnoty.
  Například:
  
      julia> p = Polynomial([1, 0, 3//2])
      + 3//2*x^2 + 1//1
  
      julia> p(-4//5)
      49//25
  

In [11]:
function (p::Polynomial{T})(x::S) where { T <: Number, S <: Number }
    return sum( p.coefficients[j] * ( x ^ (j-1) ) for j in 1:(p.degree + 1))
  end

#### Dělení polynomu polynomem
  
  Divide polynomial `p` by polynomial `q` and return the quotient and remainder.
  

In [30]:
import Base.//

function //(a::T, b::T ) where { T <: AbstractFloat }
    return a / b
end

// (generic function with 9 methods)

In [31]:
function pdiv(p::Polynomial{T}, q::Polynomial{T}) where { T <: Number }
  if q.degree == -1
    throw(ArgumentError("It is not possible to divide by 0!"))
  end
  
  x = p # copy of numerator
  res = Polynomial([0]) # initialize denumerator
  while x.degree >= q.degree 
    v = last(x.coefficients) // last(q.coefficients) # divide highest member of num / highest den
    u = Polynomial(append!(zeros(T, x.degree - q.degree), v)) # gets another member of result
    res = res + u # update result
    x = x - (u*q) # update numerator
  end

  return res, x
end

pdiv (generic function with 1 method)

In [32]:
p = Polynomial([1, 2, 4])
q = Polynomial([2, 3, 1])
r = Polynomial([5, 2, 3, 2])

println(pdiv(p, q))
println(pdiv(r, q))

(4, - 10*x^1 - 7)
(+ 2*x^1 - 3, + 7*x^1 + 11)


In [14]:
p = Polynomial([1, 2, 4])
q = Polynomial([2, 3, 1])
r = Polynomial([5, 2, 3, 2])
println(p + q) # 5 + 5 + 3
println(p + r) # 2 + 7 + 4 + 6
println(r + q) # 2 + 4 + 5 + 7


+ 5*x^2 + 5*x^1 + 3
+ 2*x^3 + 7*x^2 + 4*x^1 + 6
+ 2*x^3 + 4*x^2 + 5*x^1 + 7


In [15]:
p = Polynomial([1, 2, 4])
q = Polynomial([2, 3, 1])
println(p(5))
println(q(2))

111
12


In [16]:
p = Polynomial([1, 2, 4])
q = Polynomial([2, 3, 1])
r = Polynomial([5, 2, 3, 2])
println(p*q)
println(p*r)
println(r*q)


+ 4*x^4 + 14*x^3 + 15*x^2 + 7*x^1 + 2


+ 8*x^5 + 16*x^4 + 16*x^3 + 27*x^2 + 12*x^1 + 5
+ 2*x^5 + 9*x^4 + 15*x^3 + 17*x^2 + 19*x^1 + 10


In [17]:
zeros(Int64, 0)

Int64[]

In [18]:
p = Polynomial([1, 2, 4])
q = Polynomial([2, 3, 1])
println(p)
p = -q

+ 4*x^2 + 2*x^1 + 1


- 1*x^2 - 3*x^1 - 2

In [19]:
append!( [2, 3, 5], 5 )

4-element Vector{Int64}:
 2
 3
 5
 5

In [26]:
x = 5 // 2
y = 2 // 3
x // y

15//4